In [94]:

if "bootstrap" not in locals() or bootstrap.run:
    # path management for Python
    pythonpath, = !echo $PYTHONPATH
    if "." not in pythonpath.split(":"):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    # change into the lab directory
    # bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow "hot-reloading" of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False  # change to True re-run setup
    
!pwd
%ls

/home/amazingguni/git/fsdl-text-recognizer-2022-labs
Makefile   data/            notebooks/     text_recognizer/
README.md  environment.yml  requirements/


In [95]:
from pathlib import Path
import requests


def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename


data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

In [96]:
import gzip
import pickle


def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)

In [97]:
import torch


x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [98]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


In [99]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [100]:
x_train.ndim, y_train.ndim

(2, 1)

In [101]:
x_train[0, 0], y_train[0]

(tensor(0.), tensor(5))

In [102]:
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


In [103]:
# re-execute this cell for more samples
import random

import wandb  # just for some convenience methods that convert tensors to human-friendly datatypes

import text_recognizer.metadata.mnist as metadata # metadata module holds metadata separate from data

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # the label of the image
wandb.Image(example.reshape(*metadata.DIMS)).image  # the image itself

tensor(9)


## Defining the model

In [104]:
import math

import torch


weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [105]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [106]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis=1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [107]:
bs = 64  # batch size

xb = x_train[0:bs]  # a batch of inputs
outs = model(xb)  # outputs on that batch

print(outs[0], outs.shape)  # outputs on the first element of the batch

tensor([-2.5181, -2.1894, -2.3739, -2.1450, -2.1639, -1.8418, -2.2829, -2.9010,
        -2.5495, -2.4220], grad_fn=<SelectBackward0>) torch.Size([64, 10])


## Defineing the loss and metrics

In [108]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [109]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.1406)


In [110]:
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [111]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [112]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.3227, grad_fn=<NegBackward0>) tensor(2.3026)


In [113]:
loss = loss_func(outs, yb)

loss.backward()

In [114]:
bias.grad

tensor([-0.0322, -0.0139,  0.0268, -0.0126,  0.0098,  0.0687,  0.0012, -0.0001,
        -0.0104, -0.0372])

## Defining and running the fitting loop

In [115]:
lr = 0.5  # learning rate hyperparameter
epochs = 2  # how many epochs to train for
for epoch in range(epochs):  # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):  # in batches of size bs, so roughly n / bs of them
        start_idx = ii * bs  # we are ii batches in, each of size bs
        end_idx = start_idx + bs  # and we want the next bs entires

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad():  # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # ACHTUNG: PyTorch doesn't assume you're done with gradients
            #          until you say so -- by explicitly "deleting" them,
            #          i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()

In [116]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0799, grad_fn=<NegBackward0>) tensor(1.)


In [117]:
# re-execute this cell for more samples
idx = random.randint(0, len(x_train))
example = x_train[idx:idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(3)


## Refactoring with core `torch.nn` components

## Using `torch.nn.functional` for stateless computation

In [118]:

import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [119]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))  # should be unchanged from above!

tensor(0.0799, grad_fn=<NllLossBackward0>) tensor(1.)


## Using `torch.nn.Module` to define functions whose state is given by `torch.nn.Parameters`

In [120]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))



In [121]:
def forward(self, xb: torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()
print(model(xb)[:4])
loss = loss_func(model(xb), yb)
loss.backward()
print(model.weights.grad[::17, ::2])


tensor([[-0.3468, -0.3066,  0.1199,  0.0590,  0.0188, -0.5400,  0.6538, -0.5246,
          0.3147,  0.0372],
        [-0.5542, -0.0786,  0.0962,  0.2127, -0.0057, -0.7853,  0.0979, -0.0108,
         -0.1977,  0.1232],
        [-0.0902, -0.0947, -0.0314,  0.1709,  0.0954, -0.3542,  0.2334, -0.4118,
         -0.0040,  0.5269],
        [-0.3461, -0.1720, -0.0455,  0.1757,  0.1872, -0.7587,  0.1104, -0.2399,
          0.1718,  0.0114]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 6.7138e-03,  1.3974e-02,  7.0346e-03,  8.7323e-03,  5.4499e-03],
        [ 0.

In [122]:
print(*list(model.parameters()), sep='\n')

Parameter containing:
tensor([[ 0.0834, -0.0135, -0.0553,  ..., -0.0027, -0.0338,  0.0384],
        [ 0.0386, -0.0114,  0.0355,  ..., -0.0436,  0.0249, -0.0145],
        [-0.0152,  0.0375, -0.0030,  ..., -0.0146, -0.0636, -0.0005],
        ...,
        [ 0.0228, -0.0207,  0.0003,  ..., -0.0394,  0.0045,  0.0476],
        [-0.0376, -0.0132,  0.0597,  ...,  0.0159,  0.0091, -0.0372],
        [-0.0007, -0.0360, -0.0173,  ...,  0.0334,  0.0003, -0.0288]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [123]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs
            end_idx = start_idx + bs
            xb = x_train[start_idx:end_idx]
            yb = y_train[start_idx:end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()
fit()

In [124]:
print(accuracy(model(xb), yb))

tensor(1.)


# Refactoring intermediate `torch.nn` components: network layers, optimizers, and data handling

## Using `torch.nn.Linear` for the model definition

In [125]:
import textwrap

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")

torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [139]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)
    
    def forward(self, xb):
        return self.lin(xb)

In [140]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))

tensor(2.3002, grad_fn=<NllLossBackward0>)


In [141]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [142]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[-0.0203, -0.0242, -0.0279,  ...,  0.0171,  0.0031,  0.0149],
        [ 0.0230,  0.0005, -0.0246,  ..., -0.0177, -0.0026, -0.0008],
        [-0.0308, -0.0247,  0.0303,  ...,  0.0062,  0.0052,  0.0299],
        ...,
        [-0.0135,  0.0224,  0.0134,  ...,  0.0314,  0.0016,  0.0221],
        [-0.0208,  0.0254,  0.0290,  ...,  0.0306,  0.0060, -0.0303],
        [-0.0268,  0.0149,  0.0282,  ...,  0.0161, -0.0154, -0.0135]],
       requires_grad=True)
Parameter containing:
tensor([-0.0344,  0.0098, -0.0019,  0.0087,  0.0170,  0.0077, -0.0193, -0.0235,
        -0.0326,  0.0219], requires_grad=True)


## Applying gradients with `torch.optim.Optimizer`

In [143]:
from torch import optim

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [144]:
model = MNISTLogistic()
opt = configure_optimizer(model)

print("before training:", loss_func(model(xb), yb), sep="\n\t")

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training:", loss_func(model(xb), yb), sep="\n\t")

before training:
	tensor(2.3110, grad_fn=<NllLossBackward0>)
after training:
	tensor(0.8484, grad_fn=<NllLossBackward0>)


## Organizing data with `torch.utils.data.Dataset`

In [145]:
from text_recognizer.data.util import BaseDataset

train_ds = BaseDataset(x_train, y_train)

In [146]:
model = MNISTLogistic()
opt = configure_optimizer(model)


for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]  # xb and yb in one line!
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.8503, grad_fn=<NllLossBackward0>)


## Batching up data with `torch.utils.data.DataLoader`

In [147]:
from torch.utils.data import DataLoader


train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [148]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit

In [149]:
model = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(0.8571, grad_fn=<NllLossBackward0>)


## Swapping in another model

In [151]:
from text_recognizer.models.mlp import MLP

MLP.fit = fit

In [152]:
digits_to_9 = list(range(10))
data_config = {'input_dims': (784,), 'mapping': {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [153]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [154]:
model.fc1.weight

Parameter containing:
tensor([[ 8.7242e-03, -3.1859e-02,  9.0436e-04,  ..., -3.3780e-02,
         -3.1793e-02, -1.9702e-03],
        [-2.6624e-02,  3.3537e-02,  8.3773e-03,  ...,  1.8987e-02,
          5.9025e-03,  1.8216e-03],
        [-1.7399e-02, -1.2242e-02, -3.5148e-02,  ..., -2.4685e-02,
          2.8347e-02, -1.2276e-02],
        ...,
        [-7.3215e-03,  3.5017e-02,  3.2381e-03,  ...,  3.5709e-02,
          3.5630e-03, -2.7254e-02],
        [ 3.5428e-02, -2.5912e-02, -1.7379e-02,  ..., -9.0040e-03,
          1.1055e-02, -1.9707e-02],
        [ 9.8072e-03, -1.9874e-02, -1.4348e-06,  ..., -4.2096e-03,
          2.4742e-02, -2.9996e-02]], requires_grad=True)

In [155]:
%%time

print("before training:", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training:", loss_func(model(xb), yb))

before training: tensor(2.3085, grad_fn=<NllLossBackward0>)
after training: tensor(0.3510, grad_fn=<NllLossBackward0>)
CPU times: user 1min 49s, sys: 51 ms, total: 1min 49s
Wall time: 13.7 s


# Extragoodies: data organization, validation, and acceleration

## Making the GPU go brrrrr

In [156]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("☹️")

Tue Jan 24 10:22:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.12    Driver Version: 527.41       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   33C    P8     9W / 200W |   1895MiB /  8192MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [157]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

tensor(0.2777, device='cuda:0', grad_fn=<NllLossBackward0>)

In [158]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [159]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)

print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(0.3300, device='cuda:0', grad_fn=<NllLossBackward0>)
CPU times: user 13.3 s, sys: 892 ms, total: 14.2 s
Wall time: 14.3 s


## Adding validation data and organizing data code with a `DataModule`

In [160]:
class MNISTDataModule:
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"
    
    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir / self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url + self.filename).content
            self.path.open("wb").write(content)

    def setup(self):
        with gzip.open(self.path, "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

        x_train, y_train, x_valid, y_valid = map(
            torch.tensor, (x_train, y_train, x_valid, y_valid)
            )
        
        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_valid, y_valid, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_ds, batch_size=2 * self.bs, shuffle=False)

In [161]:
def fit(self: nn.Module, datamodule):
    datamodule.prepare_data()
    datamodule.setup()

    val_dataloader = datamodule.val_dataloader()
    
    self.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

    print("before start of training:", valid_loss / len(val_dataloader))

    opt = configure_optimizer(self)
    train_dataloader = datamodule.train_dataloader()
    for epoch in range(epochs):
        self.train()
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.eval()
        with torch.no_grad():
            valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

        print(epoch, valid_loss / len(val_dataloader))


MNISTLogistic.fit = fit
MLP.fit = fit

In [162]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3047, device='cuda:0')
0 tensor(0.1673, device='cuda:0')
1 tensor(0.1168, device='cuda:0')


# Exercises

### 🌟 Try out different hyperparameters for the `MLP` and for training.

Can you get a final `valid`ation `acc`uracy of 98%?
Can you get to 95% 2x faster than the baseline `MLP`?

In [164]:
%%time 
from argparse import Namespace  # you'll need this

args = None  # edit this

epochs = 5  # used in fit
bs = 32  # used by the DataModule


# used in fit, play around with this if you'd like
def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)


model = MLP(data_config, args=args)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

val_dataloader = datamodule.val_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

before start of training: tensor(2.3080, device='cuda:0')
0 tensor(0.1649, device='cuda:0')
1 tensor(0.1111, device='cuda:0')
CPU times: user 21.8 s, sys: 1.71 s, total: 23.5 s
Wall time: 23.2 s


tensor(0.9677, device='cuda:0')